# VITS fast-finetunning 전용 추론 코랩
다른 VITS 계열 추론 아마 안 됨

Bert Vits2는 당연히 안 됨

STEP 5로 G.pth과 json 파일이 필수입니다

## **※※※CPU로도 가능※※※**


> 코랩은 정책상 WebUI를 이용하는 것을 금지하고 있습니다. 이 코랩을 이용하는 모든 이용자는, 이 코랩을 이용하는 것으로 인해 **코랩 영구 사용 불가** 등의 불이익을 당할 수도 있다는 사실을 인지하였다고 간주합니다.



In [ ]:
# @title 준비
!git clone https://github.com/Plachtaa/VITS-fast-fine-tuning.git
!python -m pip install --upgrade --force-reinstall regex
!python -m pip install --force-reinstall soundfile
!python -m pip install --force-reinstall gradio==3.50.2
#!pip uninstall gradio --yes
#!pip install gradio==3.50.2
!python -m pip install imageio==2.4.1
!python -m pip install --upgrade youtube-dl
!python -m pip install moviepy
%cd VITS-fast-fine-tuning

!python -m pip install --no-build-isolation -r requirements.txt
!python -m pip install --upgrade numpy
!python -m pip install --upgrade --force-reinstall numba
!python -m pip install --upgrade Cython

!python -m pip install --upgrade pyzmq
!python -m pip install pydantic==1.10.4
!python -m pip install ruamel.yaml
!python -m pip install git+https://github.com/openai/whisper.git
!python -m pip install gdown

# build monotonic align
%cd monotonic_align/
!mkdir monotonic_align
!python setup.py build_ext --inplace
%cd ..
#!mkdir pretrained_models
# download data for fine-tuning
#!wget https://huggingface.co/datasets/Plachta/sampled_audio4ft/resolve/main/sampled_audio4ft_v2.zip
#!unzip sampled_audio4ft_v2.zip
# create necessary directories
#!mkdir video_data
#!mkdir raw_audio
#!mkdir denoised_audio
#!mkdir custom_character_voice
#!mkdir segmented_character_voice

!export LC_ALL="en_US.UTF-8"
!export LD_LIBRARY_PATH="/usr/lib64-nvidia"
!export LIBRARY_PATH="/usr/local/cuda/lib64/stubs"
#!ldconfig /usr/lib64-nvidia

!gdown --fuzzy https://drive.google.com/file/d/19BWvspGUMpw0wIzBLRkJCwKxh_1fvDYo/view?usp=sharing -O /content/VITS-fast-fine-tuning/VC_inference.py

Cloning into 'VITS-fast-fine-tuning'...
remote: Enumerating objects: 883, done.
remote: Counting objects: 100% (378/378), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 883 (delta 272), reused 266 (delta 228), pack-reused 505
Receiving objects: 100% (883/883), 650.94 KiB | 2.60 MiB/s, done.
Resolving deltas: 100% (507/507), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: regex
    Found existing installation: regex 2023.6.3
    Uninstalling regex-2023.6.3:
      Successfully uninstalled regex-2023.6.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.9/443.9 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: pycparser
    Found existing installation: pycparser 2.21
    Uninstalling pycparser-2.21:
      Successfully uninstalled pycparser-2.21

In [ ]:
# @title 드라이브 마운트
#@markdown 자신의 드라이브에서 모델을 다운로드할 경우 런타임당 1회 실행합니다. ('/content/drive/...')

#@markdown 자신의 드라이브라고 해도 공유 링크를 사용하여 모델을 다운로드할 경우에는 필요 없습니다. ('drive.google.com...')
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title 모델 로딩

#@markdown ###한 번에 여러 모델을 다운받아 사용할 수 없습니다.
#@markdown 한 런타임 당 하나의 모델만 사용해주세요

#@markdown Google Drive 링크 또는 Hugging Face 링크에서의 다운로드만을 지원합니다.

#@markdown WebUI를 껐다가 다른 모델을 불러오기 위해서는 아래 clean_model_folder를 체크하여 이전 모델을 삭제하여주세요.

clean_model_folder = True # @param {type:"boolean"}


import os
import shutil
import urllib.request
import gdown
from urllib.parse import urlparse, parse_qs
import urllib.parse
from google.oauth2.service_account import Credentials
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests
from requests import get
import hashlib

G_PTH_WHERE = "https://huggingface.co/spaces/FrankZxShen/vits-fast-finetuning-umamusume/resolve/main/models/G_15800.pth?download=true" # @param {type:"string"}
SPEAKER_JSON_WHERE = "https://huggingface.co/spaces/FrankZxShen/vits-fast-finetuning-umamusume/resolve/main/models/config.json?download=true" # @param {type:"string"}

if clean_model_folder is True:
  if os.path.exists('/content/VITS-fast-fine-tuning/inference_folder'):
    shutil.rmtree('/content/VITS-fast-fine-tuning/inference_folder')

os.makedirs('/content/VITS-fast-fine-tuning/inference_folder', exist_ok=True)
dir_name = 'inference_folder'

#%cd inference_folder

def download_model(url, dir_name):
    try:
        #extraction_folder = os.path.join(rvc_models_dir, dir_name)

        download_name = url.split('/')[-1]
        if '?download=true' in download_name:
          download_name = download_name.split('?')[-2]
        #download_where = '/inference_folder/' +  url.split('/')[-1]
        download_where = '/content/VITS-fast-fine-tuning/inference_folder/' +  download_name

        print(f'[~] {download_name}를 허깅페이스에서 다운받고 있어요...')


        #urllib.request.urlretrieve(url, download_where)
        #urllib.request.urlretrieve(url, '/content/VITS-fast-fine-tuning/inference_folder')

        #print('[~] Extracting zip...')
        #extract_zip(extraction_folder, download_where)
        if "/blob/" in url:
          url = url.replace("blob", "resolve")
          print("Resolved URL:", url)  # Print the resolved URL
          !wget "$url" -O "$download_where"
          #/content/VITS-fast-fine-tuning/inference_folder
        else:
          !wget "$url" -O "$download_where"
          #/content/VITS-fast-fine-tuning/inference_folder
          #urllib.request.urlretrieve(url, '/content/VITS-fast-fine-tuning/inference_folder/')
        print(f'[+] {download_name}를 허깅페이스에서 성공적으로 다운로드 했어요!')

    except Exception as e:
        raise Exception(str(e))

def download_online_model_drive(file_id, dir_name):
    try:
        download_where = '/content/VITS-fast-fine-tuning/inference_folder/'

        print(f'[~] 링크 공유된 구글드라이브에서 파일을 다운받고 있어요...')

        gdown.download(f'https://drive.google.com/uc?id={file_id}', download_where, quiet=False)

        #downloaded_where = max(glob(os.path.join(download_where, "*")), key=os.path.getctime)
        #download_where = downloaded_where.split('/')[-1]
        #print(f'[+] {download_name}를 링크 공유된 구글드라이브에서 성공적으로 다운로드 했어요!')
        print(f'[+] 링크 공유된 구글드라이브에서 파일을 성공적으로 다운로드 했어요!')

    except Exception as e:
        raise Exception(str(e))
def download_from_your_drive(url, dir_name):
    try:
        download_name = url.split('/')[-1]
        download_where = '/content/VITS-fast-fine-tuning/inference_folder/' +  url.split('/')[-1]

        print(f'[~] {download_name}를 마운트 된 구글드라이브에서 다운받고 있어요...')

        !cp {url} {download_where}

        print(f'[+] {download_name}를 마운트 된 구글드라이브에서 성공적으로 다운로드 했어요!')
    except Exception as e:
        raise Exception(str(e))

# Detecta si la URL es de Google Drive o Hugging Face
if 'drive.google.com' in G_PTH_WHERE:
    file_id_drive = G_PTH_WHERE.split('/')[-2]
    download_online_model_drive(file_id_drive, dir_name)
elif 'huggingface.co' in G_PTH_WHERE:
    download_model(G_PTH_WHERE, dir_name)
elif '/content/drive' in G_PTH_WHERE:
  download_from_your_drive(G_PTH_WHERE, dir_name)
else:
    print('모르는 URL이네요... G_PTH_WHERE가 Google Drive 링크 또는 Hugging Face 링크에서만 다운받을 수 있어요!')

if 'drive.google.com' in SPEAKER_JSON_WHERE:
    file_id_drive = SPEAKER_JSON_WHERE.split('/')[-2]
    download_online_model_drive(file_id_drive, dir_name)
elif 'huggingface.co' in SPEAKER_JSON_WHERE:
    download_model(SPEAKER_JSON_WHERE, dir_name)
elif '/content/drive' in SPEAKER_JSON_WHERE:
  download_from_your_drive(SPEAKER_JSON_WHERE, dir_name)
else:
    print('모르는 URL이네요... SPEAKER_JSON_WHERE가 Google Drive 링크 또는 Hugging Face 링크에서만 다운받을 수 있어요!')

#%cd ..


[~] G_latest.pth를 허깅페이스에서 다운받고 있어요...()
--2023-12-24 14:36:00--  https://huggingface.co/guetLzy/VITS-fast-fine-tuning/resolve/main/G_latest.pth?download=true
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.17, 18.164.174.55, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/64/e7/64e7d2dc14d36068ee87f3ac5fbdb52987957b713044e77aa3d134b9d970e2ab/2142f1b75eaf27fc2d21d5b4a0d3ae8aec1e7606bd5a770d1b73e70efd5b5c01?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27G_latest.pth%3B+filename%3D%22G_latest.pth%22%3B&Expires=1703687760&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwMzY4Nzc2MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy82NC9lNy82NGU3ZDJkYzE0ZDM2MDY4ZWU4N2YzYWM1ZmJkYjUyOTg3OTU3YjcxMzA0NGU3N2FhM2QxMzRiOWQ5NzBlMmFiLzIxNDJmMWI3NWVhZjI3ZmMyZDIxZDViNGEwZ

In [ ]:
#https://drive.google.com/file/d/1nwbG7fM5kztBFD-BG2wOcRUF0FUrEFQ3/view?usp=drive_link
#https://drive.google.com/file/d/15TUws0mxZqovwHjp3ZjFnD5PTXOMl2sG/view?usp=drive_link
#https://drive.google.com/file/d/1_ohqDjFbVJi1upxyJHMUg5kvveHo6oRe/view?usp=drive_link

#https://drive.google.com/file/d/19BWvspGUMpw0wIzBLRkJCwKxh_1fvDYo/view?usp=sharing

In [ ]:

#@markdown ### WebUI 실행
#!cp ./configs/modified_finetune_speaker.json ./finetune_speaker.json

from glob import glob
import shutil
import os.path
from os import path

download_where = '/content/VITS-fast-fine-tuning/inference_folder'

#G_where = max(glob(os.path.join(download_where, "G_*.pth")), key=os.path.getctime)
G_where = max(glob(os.path.join(download_where, "*.pth")), key=os.path.getctime)
G_ = './' + G_where.split('/')[-2] + G_where.split('/')[-1]
json_where = max(glob(os.path.join(download_where, "*.json")), key=os.path.getctime)
json_ = './' + json_where.split('/')[-1] + json_where.split('/')[-1]

#!python scripts/rearrange_speaker.py --model_dir {G_where} --config_dir {json_where}

#Model_Dir = './OUTPUT_MODEL/G_latest.pth'

#!python VC_inference.py --model_dir ./OUTPUT_MODEL/G_latest.pth --share True
!python VC_inference.py --model_dir {G_where} --config_dir {json_where} --share True
#!python VC_inference.py --model_dir {G_} --share True

DEBUG:matplotlib:(private) matplotlib data path: /usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data
DEBUG:matplotlib:matplotlib data path: /usr/local/lib/python3.10/dist-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/root/.config/matplotlib
DEBUG:matplotlib:matplotlib version 3.3.1
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', '_io', 'marshal', 'posix', '_frozen_importlib_external', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '_abc', 'abc', 'io', '__main__', 'types', 'enum', '_sre', 'sre_constants', 'sre_parse', 'sre_compile', '_collections_abc', 'itertools', 'keyword', '_operator', 'operator', 'reprlib', '_collections', 'collections', '_functools', 'functools', '_locale', 'copyreg', 're', 'warnings', '_stat', 'stat', 'genericpath', 'posixpath', 'os.path', 'os', '